In [1]:
import sys
sys.path.insert(0, '../gofher')

import os
import matplotlib.image as mpimg

from gofher import run_gofher
from visualize import visualize
from file_helper import write_csv,check_if_folder_exists_and_create
from spin_parity import read_spin_parity_galaxies_label_from_csv, standardize_galaxy_name

In [2]:
survery_to_use = "panstarrs"

BANDS_IN_ORDER = ['g','r','i','z','y'] #Important: Must stay in order of BLUEST to REDDEST Waveband (Editting this will cause gofher to no longer correctly evaluate redder side of galaxy)
REF_BANDS_IN_ORDER = ['i','z','y','r','g'] #The prefernce each waveband being choosen as refernce band from highest priority to lowest priority

In [3]:
figure_to_run_on = "figure11"
bin_size = 4 #None or a positive integer

In [4]:
generate_verbose_csv = True
generate_ebm_csv = True
generate_params_csv = True
generate_visualization = True
save_visualization = True

In [5]:
#Important: Make sure you update these values:
path_to_catalog_data = "..\\..\\spin-parity-catalog-data"
path_to_output = "..\\..\\gofher-data\\panstarrs\\source_extraction"

In [6]:
def get_fits_path(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_{}.fits".format(name,band))

def get_color_image_path(name):
    return os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on,name,"{}_color.jfif".format(name))

def get_path_to_catalog_csv():
    return os.path.join(path_to_catalog_data,"catalog","{}.csv".format(figure_to_run_on))

In [7]:
def get_paper_dark_side_labels():
    return read_spin_parity_galaxies_label_from_csv(get_path_to_catalog_csv())

def get_galaxies():
    return os.listdir(os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on))

In [8]:
def run_gofher_on_catalog():
    paper_labels = get_paper_dark_side_labels()

    verbose_header = []
    verbose_rows = []

    ebm_header = []
    ebm_rows = []

    params_header = []
    params_rows = []

    i = 1


    for name in get_galaxies():

        if standardize_galaxy_name(name) not in paper_labels:
            print("skippimg",name)
            continue

        print(name, i,"of",len(get_galaxies()))

        try:
            paper_label = paper_labels[standardize_galaxy_name(name)]
            gal = run_gofher(name,get_fits_path,BANDS_IN_ORDER,REF_BANDS_IN_ORDER, paper_label,s=bin_size,bin_prior_to_param_fitting=True)

            if generate_verbose_csv:
                (header,row) = gal.get_verbose_csv_header_and_row(BANDS_IN_ORDER,paper_label)
                if len(verbose_header) == 0: verbose_header = header
                verbose_rows.append(row)

            if generate_ebm_csv:
                (header,row) = gal.get_ebm_csv_header_and_row(BANDS_IN_ORDER, paper_label)
                if len(ebm_header) == 0: ebm_header = header
                ebm_rows.append(row)

            if generate_params_csv:
                (header,row) = gal.get_params_csv_header_and_row()
                if len(params_header) == 0: params_header = header
                params_rows.append(row)

            if generate_visualization:
                save_path = ''
                
                if save_visualization:
                    sub_folder = os.path.join(path_to_output,figure_to_run_on)
                    check_if_folder_exists_and_create(sub_folder)
                    save_path = os.path.join(sub_folder,"{}.png".format(name))

                color_image = color = mpimg.imread(get_color_image_path(name))
                visualize(gal,color_image,BANDS_IN_ORDER,paper_label,save_path=save_path)
        except Exception as e:
            print(e)
        i += 1

    if generate_verbose_csv:
        verbose_csv_path = os.path.join(path_to_output,"{}_verbose.csv".format(figure_to_run_on))
        write_csv(verbose_csv_path,verbose_header,verbose_rows)

    if generate_ebm_csv:
        ebm_csv_path = os.path.join(path_to_output,"{}_ebm.csv".format(figure_to_run_on))
        write_csv(ebm_csv_path,ebm_header,ebm_rows)

    if generate_params_csv:
        params_csv_path = os.path.join(path_to_output,"{}_params.csv".format(figure_to_run_on))
        write_csv(params_csv_path,params_header,params_rows)

In [9]:
if not os.path.exists(path_to_catalog_data):
    raise ValueError("The path to the catalog is not found {} - make sure you update path_to_catalog_data".format(path_to_catalog_data))

if not os.path.exists(path_to_output):
    raise ValueError("The path output is not found {} - make sure you update ppath_to_output".format(path_to_catalog_data))


run_gofher_on_catalog()

IC2247 1 of 25
IC540 2 of 25
IC944 3 of 25
MCG-02-02-040 4 of 25
MCG-02-03-015 5 of 25
NGC1542 6 of 25
NGC3067 7 of 25
NGC3079 8 of 25
NGC3169 9 of 25
NGC3495 10 of 25
NGC3626 11 of 25
NGC4517 12 of 25
NGC4605 13 of 25
NGC4772 14 of 25
NGC6314 15 of 25
NGC681 16 of 25
UGC10205 17 of 25
UGC10297 18 of 25
UGC3107 19 of 25
UGC5111 20 of 25
UGC5498 21 of 25
UGC6036 22 of 25
UGC8267 23 of 25
UGC8778 24 of 25
UGC9665 25 of 25
